# Objective
This research has two objectives:
1. Replicate the results of the reference paper **"Inside insider trading: Patterns & discoveries from a large scale exploratory analysis"** by A. Tamersoy et al. [1](https://poloclub.github.io/polochau/insider/asonam13_insider.pdf) on 150 random S&P 500 companies from 1993 to December 2020. Insider trading data will be analysed from temporal and network-centric aspects.
2. Produce a data pipeline to automate the scraping and cleaning of Form 4 data as much as possible. Large-scale analysis of Form 4 data only began in 2012 by the reference paper.

In [1]:
# Essential imports and constants
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import sqlite3 as sql
from bs4 import BeautifulSoup as bs
import requests, time, datetime, re

# if executed locally
TEST = 'Rationale/test.csv'
TICKER = 'Metadata/ticker.txt'
FORM4_LINK = 'Database/form4_data.csv'
FORM4_LINK_XML = 'Database/form4_data_xml.csv'
FORM4_LINK_HTM = 'Database/form4_data_htm.csv'
FORM4_LINK_TXT = 'Database/form4_data_txt.csv'
FORM4_DETAIL = 'Database/form4_detail.csv' 
FORM4_DETAIL_XML = 'Database/form4_detail_xml.csv'
FORM4_DETAIL_HTM = 'Database/form4_detail_htm.csv'
FORM4_DETAIL_TXT = 'Database/form4_detail_txt.csv'
FORM4_TX = 'Database/form4_tx.csv'
FORM4_TX_XML = 'Database/form4_tx_xml.csv'
FORM4_TX_HTM = 'Database/form4_tx_htm.csv'
FORM4_TX_TXT = 'Database/form4_tx_txt.csv'
META_DJ30 = 'Metadata/ticker_dj30.txt' 
META_SP500 = 'Metadata/ticker_sp500.txt'
META_SP500_RAN = 'Metadata/ticker_sp500_ran.txt'
CODE_EXTRACT = 'DataProcessingPipelining/extract_file.sh'
CODE_EXTRACT_ONE = 'DataProcessingPipelining/extract_1_quarter.sh'
CODE_EXTRACT_NOPAR = 'DataProcessingPipelining/extract_noPAR.sh'
LOG_LINK = 'Database/extract_link_log.txt'
LOG_NETWORK = 'Database/network_log.txt'

In [ ]:
# if executed on Google Colab
from google.colab import drive
drive.mount('/content/drive')

BASE_DRIVE_LINK = '/content/drive/My Drive/URECA/'
FORM4_LINK = BASE_DRIVE_LINK + 'Database/form4_data.csv'
FORM4_LINK_XML = BASE_DRIVE_LINK + 'Database/form4_data_xml.csv'
FORM4_LINK_HTM = BASE_DRIVE_LINK + 'Database/form4_data_htm.csv'
FORM4_LINK_TXT = BASE_DRIVE_LINK + 'Database/form4_data_txt.csv'
FORM4_DETAIL = BASE_DRIVE_LINK + 'Database/form4_detail.csv'
FORM4_DETAIL_XML = BASE_DRIVE_LINK + 'Database/form4_detail_xml.csv'
FORM4_DETAIL_HTM = BASE_DRIVE_LINK + 'Database/form4_detail_htm.csv'
FORM4_DETAIL_TXT = BASE_DRIVE_LINK + 'Database/form4_detail_txt.csv'
FORM4_TX = BASE_DRIVE_LINK + 'Database/form4_tx.csv'
FORM4_TX_XML = BASE_DRIVE_LINK + 'Database/form4_tx_xml.csv'
FORM4_TX_HTM = BASE_DRIVE_LINK + 'Database/form4_tx_htm.csv'
FORM4_TX_TXT = BASE_DRIVE_LINK + 'Database/form4_tx_txt.csv'
META_DJ30 = BASE_DRIVE_LINK + 'Metadata/ticker_dj30.txt' 
META_SP500 = BASE_DRIVE_LINK + 'Metadata/ticker_sp500.txt'
META_SP500_RAN = BASE_DRIVE_LINK + 'Metadata/ticker_sp500_ran.txt'
TEST = BASE_DRIVE_LINK + 'Rationale/test.csv'
TICKER = BASE_DRIVE_LINK + 'Metadata/ticker.txt'
CODE_EXTRACT = BASE_DRIVE_LINK + 'DataProcessingPipelining/extract_file.sh'
CODE_EXTRACT_ONE = BASE_DRIVE_LINK + 'DataProcessingPipelining/extract_1_quarter.sh'
CODE_EXTRACT_NOPAR = BASE_DRIVE_LINK + 'DataProcessingPipelining/extract_noPAR.sh'
LOG_LINK = BASE_DRIVE_LINK + 'Database/extract_link_log.txt'
LOG_NETWORK = BASE_DRIVE_LINK + 'Database/network_log.txt'

In [6]:
# snippet to download files
from google.colab import files
files.download('<filename here>')  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Methodology

## Rationale for Using GNU Parallel
Experimenting with GNU Parallel. For SEC EDGAR file, only 10 requests per second are allowed. To test GNU Parallel, https://httpbin.org/#/Dynamic_data is used. 10 API calls that will delay 5 seconds are launched by curl in parallel. The results are obtained in 6s, which is a good estimate that GNU parallel is functioning.

In [ ]:
%%bash
seq 10 | time parallel -j10 'curl -X GET "https://httpbin.org/delay/5" -H "accept: application/json"'

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {}, 
  "headers": {
    "Accept": "application/json", 
    "Host": "httpbin.org", 
    "User-Agent": "curl/7.68.0", 
    "X-Amzn-Trace-Id": "Root=1-60365028-2c22436a65bb09cd017d013f"
  }, 
  "origin": "155.69.175.63", 
  "url": "https://httpbin.org/delay/5"
}
{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {}, 
  "headers": {
    "Accept": "application/json", 
    "Host": "httpbin.org", 
    "User-Agent": "curl/7.68.0", 
    "X-Amzn-Trace-Id": "Root=1-60365028-6b55be27021de3d366bd4b5c"
  }, 
  "origin": "155.69.175.63", 
  "url": "https://httpbin.org/delay/5"
}
{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {}, 
  "headers": {
    "Accept": "application/json", 
    "Host": "httpbin.org", 
    "User-Agent": "curl/7.68.0", 
    "X-Amzn-Trace-Id": "Root=1-60365028-5e141fe6713b019911066cc7"
  }, 
  "origin": "155.69.175.63", 
  "url": "https://httpbin.org/delay/5"
}
{
  "args": {}, 
  "data": "", 
  "files": {},

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   318  100   318    0     0     52      0  0:00:06  0:00:06 --:--:--    78
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (7) Failed to connect to 1 port 80: Connection refused
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   318  100   318    0     0     52      0  0:00:06  0:00:06 --:--:--    78
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:

CalledProcessError: Command 'b'seq 10 | time parallel -j10 \'curl -X GET "https://httpbin.org/delay/5" -H "accept: application/json"\'\n'' returned non-zero exit status 10.

As an extension, API calls that will have delays from 1 to 15 seconds are made. With GNU Parallel, curl obtains the data in 18 seconds, showing that it does exhibit parallel behaviour.

In [ ]:
%%bash
seq 15 | time parallel -j10 'curl -X GET "https://httpbin.org/delay/{}" -H "accept: application/json" -N'

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {}, 
  "headers": {
    "Accept": "application/json", 
    "Host": "httpbin.org", 
    "User-Agent": "curl/7.68.0", 
    "X-Amzn-Trace-Id": "Root=1-60364d76-294f327a183efe5e4fddec92"
  }, 
  "origin": "155.69.175.63", 
  "url": "https://httpbin.org/delay/1"
}
{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {}, 
  "headers": {
    "Accept": "application/json", 
    "Host": "httpbin.org", 
    "User-Agent": "curl/7.68.0", 
    "X-Amzn-Trace-Id": "Root=1-60364d76-751ac2a11a21f0ec69fdf1b4"
  }, 
  "origin": "155.69.175.63", 
  "url": "https://httpbin.org/delay/2"
}
{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {}, 
  "headers": {
    "Accept": "application/json", 
    "Host": "httpbin.org", 
    "User-Agent": "curl/7.68.0", 
    "X-Amzn-Trace-Id": "Root=1-60364d76-5c87682d567d70e5040f4282"
  }, 
  "origin": "155.69.175.63", 
  "url": "https://httpbin.org/delay/3"
}
{
  "args": {}, 
  "data": "", 
  "files": {},

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   318  100   318    0     0    155      0  0:00:02  0:00:02 --:--:--   155
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   318  100   318    0     0    105      0  0:00:03  0:00:03 --:--:--   105
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   318  100   318    0     0     78      0  0:00:04  0:00:04 --:--:--    78
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   318  100   318    0     0     62      0  0:00:05  0:00:05 --:--:--    78
  % Total    % Received % Xferd  Average Speed   Tim

`curl` can be configured to not buffer the outputs while the execution is ongoing with `-N` option

In [ ]:
%%bash
curl -X GET "https://httpbin.org/drip?duration=2&numbytes=10&code=200&delay=2" -H "accept: application/octet-stream" -N

**********

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    10  100    10    0     0      2      0  0:00:05  0:00:04  0:00:01     2


## Rationale for Using Scripting
If Form 4 files from 1993 to 2021 are all downloaded without processing on-the-fly, files will be at least 2GB. However, not all data are needed for data processing. To extract the right data for analysis, Bash script is used. 
  
The typical way of handling data would be to read in the data files and process them by using Pandas. However, this process may not be suitable for large files due to memory and time complexities (paging will create bottleneck in bringing into the physical memory)

*Note: in this research, on-the-fly processing is carried out via Bash Script (see Section 3). This section is relevant for future studies that attempt to keep the files.*

The script below generates a csv file with 2.55GB data, where there are 23 columns and 50 million rows of data. Each row of data is 51 bytes. The file size is the typical file size of all Form 4 files in SEC EDGAR

In [ ]:
%%bash
# Add header for easier reference
echo "digit_1,mul2,digit_2,digit_3,digit_4,digit_5,mod3,mod5,doubledigit_1,digit_6,digit_7,digit_8,mod4,doubledigit_2,digit_9,digit_10,digit_11,mod7,triple,doubledigit_3,digit12,digit13,digit14" | tee test.csv
for i in $(seq 1 1 50000000); do 
    tmp=$(($i%10));
    echo $tmp,$(($(($i*2))%10)),$tmp,$tmp,$tmp,$tmp,$(($tmp % 3)),$(($tmp % 5)),$tmp$tmp,$tmp,$tmp,$tmp,$(($tmp % 4)),$tmp$tmp,$tmp,$tmp,$tmp,$(($tmp % 7)),$tmp$tmp$tmp,$tmp$tmp,$tmp,$tmp,$(($tmp % 4)); 
done | tee -a test.csv | awk 'FNR < 15'

# using awk instead of head as it is asynchronous [https://unix.stackexchange.com/questions/47932/how-do-i-use-tee-to-redirect-to-grep]
# command for awk [https://www.unix.com/shell-programming-and-scripting/82416-printing-first-n-lines-file-without-using-head.html]

digit_1,mul2,digit_2,digit_3,digit_4,digit_5,mod3,mod5,doubledigit_1,digit_6,digit_7,digit_8,mod4,doubledigit_2,digit_9,digit_10,digit_11,mod7,triple,doubledigit_3,digit12,digit13,digit14
1,2,1,1,1,1,1,1,11,1,1,1,1,11,1,1,1,1,111,11,1,1,1
2,4,2,2,2,2,2,2,22,2,2,2,2,22,2,2,2,2,222,22,2,2,2
3,6,3,3,3,3,0,3,33,3,3,3,3,33,3,3,3,3,333,33,3,3,3
4,8,4,4,4,4,1,4,44,4,4,4,0,44,4,4,4,4,444,44,4,4,0
5,0,5,5,5,5,2,0,55,5,5,5,1,55,5,5,5,5,555,55,5,5,1
6,2,6,6,6,6,0,1,66,6,6,6,2,66,6,6,6,6,666,66,6,6,2
7,4,7,7,7,7,1,2,77,7,7,7,3,77,7,7,7,0,777,77,7,7,3
8,6,8,8,8,8,2,3,88,8,8,8,0,88,8,8,8,1,888,88,8,8,0
9,8,9,9,9,9,0,4,99,9,9,9,1,99,9,9,9,2,999,99,9,9,1
0,0,0,0,0,0,0,0,00,0,0,0,0,00,0,0,0,0,000,00,0,0,0
1,2,1,1,1,1,1,1,11,1,1,1,1,11,1,1,1,1,111,11,1,1,1
2,4,2,2,2,2,2,2,22,2,2,2,2,22,2,2,2,2,222,22,2,2,2
3,6,3,3,3,3,0,3,33,3,3,3,3,33,3,3,3,3,333,33,3,3,3
4,8,4,4,4,4,1,4,44,4,4,4,0,44,4,4,4,4,444,44,4,4,0


Consider loading the whole dataset into csv file just to get those rows with 'triple' = 111

For Pandas, the naive approach would be to load the whole dataset into a variable `df`, and then use Boolean operations and masking to obtain the required data.

In [ ]:
%%time
data = pd.read_csv(TEST, header = 0)
df = data.loc[data['triple']==111, :]
display(df)

MemoryError: Unable to allocate 8.57 GiB for an array with shape (23, 50000000) and data type int64

`MemoryError` occured as the physical memory on the local machine is insufficient for the whole dataset. An approach to circumvent this issue is to load the file in chunks or to use appropriate data types to add

In [ ]:
a = datetime.datetime.now()
df_full = pd.DataFrame()
for chunk in pd.read_csv(TEST, header = 0, chunksize = 100000):
    df = chunk.loc[(chunk['triple'] == 111), :]
    df_full = pd.concat([df_full, df], axis = 0)
b = datetime.datetime.now()
display(df_full)
print("Total Time Taken:", (b-a).total_seconds())

,digit_1,mul2,digit_2,digit_3,digit_4,digit_5,mod3,mod5,doubledigit_1,digit_6,...,doubledigit_2,digit_9,digit_10,digit_11,mod7,triple,doubledigit_3,digit12,digit13,digit14
0,1,2,1,1,1,1,1,1,11,1,...,11,1,1,1,1,111,11,1,1,1
10,1,2,1,1,1,1,1,1,11,1,...,11,1,1,1,1,111,11,1,1,1
20,1,2,1,1,1,1,1,1,11,1,...,11,1,1,1,1,111,11,1,1,1
30,1,2,1,1,1,1,1,1,11,1,...,11,1,1,1,1,111,11,1,1,1
40,1,2,1,1,1,1,1,1,11,1,...,11,1,1,1,1,111,11,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49999950,1,2,1,1,1,1,1,1,11,1,...,11,1,1,1,1,111,11,1,1,1
49999960,1,2,1,1,1,1,1,1,11,1,...,11,1,1,1,1,111,11,1,1,1
49999970,1,2,1,1,1,1,1,1,11,1,...,11,1,1,1,1,111,11,1,1,1
49999980,1,2,1,1,1,1,1,1,11,1,...,11,1,1,1,1,111,11,1,1,1


Total Time Taken: 131.005842


To process 2.5GB of data, Pandas took approximately 2 minutes 10 seconds (131.005842 seconds). Consider using bash script with awk, which streams and processes data line by line

In [ ]:
%%bash
time awk 'BEGIN {FS=","} {if (($19 == 111))
    print $0;
}' 'Rationale/test.csv' >buffer.csv


real	0m45.338s
user	0m42.406s
sys	0m2.859s


In [ ]:
a = datetime.datetime.now()
df_all_bash = pd.read_csv('buffer.csv', header = 0)
display(df_all_bash)
b = datetime.datetime.now()
print((b-a).total_seconds())

,1,2,1.1,1.2,1.3,1.4,1.5,1.6,11,1.7,...,11.1,1.11,1.12,1.13,1.14,111,11.2,1.15,1.16,1.17
0,1,2,1,1,1,1,1,1,11,1,...,11,1,1,1,1,111,11,1,1,1
1,1,2,1,1,1,1,1,1,11,1,...,11,1,1,1,1,111,11,1,1,1
2,1,2,1,1,1,1,1,1,11,1,...,11,1,1,1,1,111,11,1,1,1
3,1,2,1,1,1,1,1,1,11,1,...,11,1,1,1,1,111,11,1,1,1
4,1,2,1,1,1,1,1,1,11,1,...,11,1,1,1,1,111,11,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999994,1,2,1,1,1,1,1,1,11,1,...,11,1,1,1,1,111,11,1,1,1
4999995,1,2,1,1,1,1,1,1,11,1,...,11,1,1,1,1,111,11,1,1,1
4999996,1,2,1,1,1,1,1,1,11,1,...,11,1,1,1,1,111,11,1,1,1
4999997,1,2,1,1,1,1,1,1,11,1,...,11,1,1,1,1,111,11,1,1,1


6.468974


Adding the processing and loading time of the script approach yields less than 55 seconds, compared with 131 seconds by using Pandas approach. The script approach takes less than 45% of the time taken by Pandas approach.

This quick example illustrates the need for pre-filtering of data instead of loading the whole dataset into the physical memory. Although there are other variables that influence the ability to load DataFrames (eg. data types used to store each datum, number of processors on the machine), this example is intended to just explain the rationale of using scripting to filter data

An alternative method is to store the data in SQL database and filter the necessary data before reading them. The following example shows this approach by using SQLite, a small, portable and full-featured database engine. SQLite is used in this example for two reasons:
1. For easy replication of this research
2. It has lower operational overhead than other database engines such as MySQL and PostgreSQL

In [ ]:
%%time
conn = sql.connect('rationale_use_script.db')

for chunk in pd.read_csv('Rationale/test.csv', header = 0, chunksize = 1000000):
    chunk.to_sql('rationale_use_script', conn, index=False, if_exists = 'append')

Wall time: 7min 33s


In [ ]:
# Verify that all data are transferred
query = """
SELECT COUNT(*)
FROM rationale_use_script;
"""
count = pd.read_sql_query(query, conn)
print(count)

   COUNT(*)
0  50000000


In [ ]:
import datetime
a = datetime.datetime.now()
query = """
SELECT *
FROM rationale_use_script
WHERE triple=111;
"""
df_full_sql = pd.read_sql_query(query, conn)
display(df_full_sql)
b = datetime.datetime.now()
print("Total Time Taken:", (b-a).total_seconds())

,digit_1,mul2,digit_2,digit_3,digit_4,digit_5,mod3,mod5,doubledigit_1,digit_6,...,doubledigit_2,digit_9,digit_10,digit_11,mod7,triple,doubledigit_3,digit12,digit13,digit14
0,1,2,1,1,1,1,1,1,11,1,...,11,1,1,1,1,111,11,1,1,1
1,1,2,1,1,1,1,1,1,11,1,...,11,1,1,1,1,111,11,1,1,1
2,1,2,1,1,1,1,1,1,11,1,...,11,1,1,1,1,111,11,1,1,1
3,1,2,1,1,1,1,1,1,11,1,...,11,1,1,1,1,111,11,1,1,1
4,1,2,1,1,1,1,1,1,11,1,...,11,1,1,1,1,111,11,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,1,2,1,1,1,1,1,1,11,1,...,11,1,1,1,1,111,11,1,1,1
4999996,1,2,1,1,1,1,1,1,11,1,...,11,1,1,1,1,111,11,1,1,1
4999997,1,2,1,1,1,1,1,1,11,1,...,11,1,1,1,1,111,11,1,1,1
4999998,1,2,1,1,1,1,1,1,11,1,...,11,1,1,1,1,111,11,1,1,1


Total Time Taken: 129.08036


Notice that the SQLite approach takes approximately 2 minutes 9 seconds (129.08036 seconds). 
  
In summary, 

|Approach | Time (rounded to nearest sec)| Approximate Ratio
--- | --- | ---
CSV | 131 | 2.52
Script | 52 | 1
SQL | 129 | 2.48

# Data Collection
To obtain data, three major actions are taken:  
1.	Identify the storage of Form 4 files in EDGAR archive to design a scraping plan.
2.	Determine the target companies to explore and scrape.
3.	Scrape the information from the Form 4 files.

## Scraping XML Links
Insider trading filings are kept in US Securities and Exchange Commission (SEC) in Electronic Data Gathering, Analysis, and Retrieval system (EDGAR), the primary system for companies and other entities to submit documents [1](https://www.sec.gov/edgar/about). Since all insider trading activities must be reported to SEC EDGAR via Form 4, insider trading analysis can be performed by scraping the website.
  
SEC EDGAR website compiles data in the following way:
- The Form 4 files since 1993 are stored in SEC EDGAR. 
- The Form 4 files are compiled into a folder each quarter of the year. To locate all of these files, EDGAR uses an index `idx` file to track the reporting companies, dates and links regarding each Form 4 file.
Hence, to quicken the data collection process, the index files will be scraped by substituting the year and quarter into the link, and obtaining the link by cURL utility. The website link has the pattern   
`https://www.sec.gov/Archives/edgar/full-index/[filing year]/QTR[filing quarter]/form.idx`
  
However, SEC EDGAR has a traffic limit of 10 requests per second. Exceeding traffic limit will cause IP block for 10 minutes. While the offered traffic limit is decent, it takes around 500 seconds to scrap just one quarter (if the quarter is the first or second quarter). Parallel execution is preferred.
  
Scraping of XML files is done in `extract.sh`. Note that a string parameter containing CIKs separated by '|' is passed into the file for filtering.

In [ ]:
%%bash -s "$CODE_EXTRACT"
cat $1

#!/bin/bash
read CIKs;

scrapDetailsFromLink() {
    temp=$(echo $4 | sed -e 's/-//g');
    xml_file=$(curl -s $4 | sed -ne '0,/<FILENAME>/s/<FILENAME>\(.*\)/\1/p');
    if [[ $? == 0 ]]; then
        new_link=$(echo $temp | sed -e "s|.txt|/$xml_file|");
        printf "%s\b|%s|%s|%s\n" "$1" $2 $3 $new_link;
    else
        printf "%s\b|%s|%s|%s\n" "$1" $2 $3 $temp;
    fi
    sleep 1;
}
export -f scrapDetailsFromLink

for i in $(seq 1993 1 2020); 
do
    for j in $(seq 1 1 4);
    do
        idx_file="https://www.sec.gov/Archives/edgar/full-index/$i/QTR$j/form.idx"
        if curl -o /dev/null --silent --fail --head $idx_file; then
            echo "Reading idx file for $i-QTR$j"
            SECONDS=0;
            dir_date=$(echo $idx_file | sed -rne "s|.*([0-9]{4})/QTR([1-4]).*|\1-QTR\2|p")
            curl -s $idx_file | grep -E "^4[[:space:]]" | grep -Ew $CIKs |
            awk -v home_link="https://www.sec.gov/Archives/" 'BEGIN{OFS="::"; ORS="\n"}
                {for(i=2;i<NF-2;

## Determine the Target Companies
This exploration focuses on **150 S&P 500 companies.** 
- As each company has a corresponding Central Index Key (CIK) and a Ticker Symbol, [ticker data](https://www.sec.gov/include/ticker.txt) is obtained from SEC EDGAR to obtain information about these companies.
- To determine a clearer company domain to explore, Dow Jones 30 and S&P 500 companies are explored
    - Dow Jones 30 companies are obtained from [Wikipedia](https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average)
    - S&P 500 companies are obtained from [Wikipedia](https://en.wikipedia.org/wiki/List_of_S%26P_500_companies)

In [ ]:
# Obtain ticker data
ticker_cik = pd.read_table(TICKER, names = ['Ticker', 'CIK'])
ticker_cik.Ticker = ticker_cik.apply(lambda x: str(x.Ticker).upper(), axis = 1)
ticker_cik.set_index('Ticker', inplace = True)

### Dow Jones 30

In [ ]:
dj_companies = pd.read_html('https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average')[1]
dj_companies = dj_companies.loc[:,['Company', 'Symbol']].set_index('Symbol')

In [ ]:
!curl -s https://www.sec.gov/include/ticker.txt >ticker.txt

In [ ]:
dj_companies = dj_companies.join(ticker_cik, how = 'inner')
grep_pat = '|'.join(dj_companies.CIK.astype(str).to_list())
with open(META_DJ30, 'w') as f:
    f.write("%s" % grep_pat)
print(grep_pat)
dj_companies.head()

320193|318154|4962|12927|18230|1108524|858877|93410|1744489|1751788|886982|354950|773840|51143|50863|200406|19617|21344|63908|66740|310158|789019|320187|80424|86312|731766|1403161|732712|1618921|104169


,Company,CIK
AAPL,Apple Inc.,320193
AMGN,Amgen,318154
AXP,American Express,4962
BA,Boeing,12927
CAT,Caterpillar Inc.,18230


### S&P 500

In [ ]:
sp_companies = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp_companies = sp_companies.loc[:, ['Symbol', 'Security']].set_index('Symbol')
sp_companies = sp_companies.merge(ticker_cik, left_index = True, right_index = True)

In [ ]:
grep_pat = '|'.join(sp_companies.CIK.astype(str).to_list())
with open(META_SP500, 'w') as f:
    f.write("%s" % grep_pat)

In [ ]:
sp_companies_random = sp_companies.sample(n=150)
grep_pat = '|'.join(sp_companies_random.CIK.astype(str).to_list())
with open(META_SP500_RAN, 'w') as f:
    f.write("%s" % grep_pat)

In this research, 150 random companies on S&P500 are chosen.

In [ ]:
%%bash -s "$META_SP500" "CODE_EXTRACT"
cat $1 | sh $2

cat: Metadata/ticker_sp500_ran.txt: No such file or directory
sh: 0: Can't open extract.sh


### Linking to 3.1: Shortening the Data Processing and Pipelining Process
GNU Parallel is used to improve the existing `extract.sh` before execution. However, as traffic limit must be considered, the total request-response time is measured. To measure this, the duration for a file is first measured. This approach and command is illustrated by [Joseph Scott](https://blog.josephscott.org/2011/10/14/timing-details-with-curl/):  
- Use the request-response information to obtain time-related measurements and output into `curl-format.txt`
- Calculate the time with `-w` option  
An example is shown below

In [ ]:
%%bash
wget http://ftp.gnu.org/gnu/parallel/parallel-20210522.tar.bz2
sudo tar xjf parallel-20210522.tar.bz2
cd parallel-20210522
sudo ./configure && make
sudo make install
cd

In [ ]:
%%bash 
curl -w "@curl-format.txt" -o /dev/null -s "https://www.sec.gov/Archives/edgar/data/320193/000032019320000101/wf-form4_160565582610158.xml"

     time_namelookup:  0.026844s
        time_connect:  0.033290s
     time_appconnect:  0.062047s
    time_pretransfer:  0.062207s
       time_redirect:  0.000000s
  time_starttransfer:  0.082111s
                     ----------
          time_total:  0.082261s


Assuming an upper limit of 0.1s for each call. Hence, within 1 second, 10 requests can be launched -- the **maximum** traffic limit of SEC EDGAR. GNU Parallel can be run, but it needs to be a delay for at least 0.92 seconds for each job. For easier coding, a delay of 1 second is used. `extract.sh` is displayed in the cell above, under section 3.1

In [ ]:
%%bash -s "$CODE_EXTRACT_ONE"
cat $1

#!/bin/bash
read CIKs;
user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"

scrapDetailsFromLink() {
    temp=$(echo $4 | sed -e 's/-//g');
    xml_file=$(curl -s $4 | sed -ne '0,/<FILENAME>/s/<FILENAME>\(.*\)/\1/p');
    if [[ $? == 0 ]]; then
        new_link=$(echo $temp | sed -e "s|.txt|/$xml_file|");
        printf "%s\b|%s|%s|%s\n" "$1" $2 $3 $new_link;
    else
        printf "%s\b|%s|%s|%s\n" "$1" $2 $3 $temp;
    fi
    sleep 1;
}
export -f scrapDetailsFromLink

for i in $(seq 2010 1 2010); 
do
    for j in $(seq 1 1 1);
    do
        idx_file="https://www.sec.gov/Archives/edgar/full-index/$i/QTR$j/form.idx"
        if curl -A $user_agent -o /dev/null --silent --fail --head $idx_file; then
            echo "Reading idx file for $i-QTR$j"
            SECONDS=0;
            dir_date=$(echo $idx_file | sed -rne "s|.*([0-9]{4})/QTR([1-4]).*|\1-QTR\2|p")
            curl -s $idx_file | grep -E "^4[[:space

In [ ]:
%%bash -s "$META_DJ30" "$CODE_EXTRACT_ONE"
cat $1 | bash $2

HTTP/2 200 
content-type: application/octet-stream
etag: "b978e6511ba0ba13b75e575b0ec94c15"
last-modified: Sat, 15 May 2021 06:24:22 GMT
server: AmazonS3
x-amz-id-2: xsnvaf+pL/5sXj7X7JXa2QuF2c7R/EhRvq6v5jMy3Mt3ZZCP0Mz5jFVH4Den1BNSZbewWmZkuNg=
x-amz-meta-mode: 33188
x-amz-meta-s3cmd-attrs: uid:504/gname:fitrprnt/uname:fitrprnt/gid:504/mode:33152/mtime:1621059776/atime:1621059780/md5:b978e6511ba0ba13b75e575b0ec94c15/ctime:1621059860
x-amz-replication-status: COMPLETED
x-amz-request-id: 505XTV9G5DAG6N37
x-amz-version-id: u8XBKKZqeNrnmM2.uBClSAwb1iZLiCh2
x-content-type-options: nosniff
x-frame-options: SAMEORIGIN
x-xss-protection: 1; mode=block
date: Sun, 16 May 2021 06:33:28 GMT
strict-transport-security: max-age=31536000 ; includeSubDomains ; preload

Reading idx file for 2010-QTR1
Runtime for 2010-QTR1: 139
./trading_2010-QTR1.csv is appended into database document


In [ ]:
%%bash -s "$META_DJ30" "$CODE_EXTRACT_ONE"
cat $1 | bash $2

Reading idx file for 2010-QTR1
Runtime for 2010-QTR1: 138
./trading_2010-QTR1.csv is appended into database document


In [ ]:
%%bash
wc -l form4_data.csv

1146 form4_data.csv


In [ ]:
%%bash -s "$CODE_EXTRACT_NOPAR"
cat $1

#!/bin/sh
read CIKs;
for i in $(seq 2010 1 2010); 
do
    for j in $(seq 1 1 1);
    do
        idx_file="https://www.sec.gov/Archives/edgar/full-index/$i/QTR$j/form.idx"
        if curl -o /dev/null --silent --fail --head $idx_file; then
            echo "Reading idx file for $i-QTR$j"
            SECONDS=0;
            dir_date=$(echo $idx_file | sed -rne "s|.*([0-9]{4})/QTR([1-4]).*|\1-QTR\2|p")
            curl -s $idx_file | grep -E "^4[[:space:]]" | grep -Ew $CIKs |
            awk -v home_link="https://www.sec.gov/Archives/" 'BEGIN{OFS=":"; ORS="\n"}
                {for(i=2;i<NF-2;i++) printf("%s ", $(i))
                print "", $(NF-2), $(NF - 1), home_link$(NF)}' |
            while IFS=: read -r company cik date link; do
                sleep 0.1000
                xml_file=`curl -s $link | sed -ne '0,/<FILENAME>/s/<FILENAME>\(.*\)/\1/p' `
                temp=`echo $link | sed -e 's/-//g'`
                new_link=`echo $temp | sed -e "s|.txt|/$xml_file|"`
               

In [ ]:
%%bash -s "$META_DJ30" "$CODE_EXTRACT_NOPAR"
cat $1 | bash $2

Reading idx file for 2010-QTR1
Runtime: 748
./trading_2010-QTR1.csv is appended into database document


In [ ]:
%%bash
wc -l data.csv

1146 data.csv


Based on the output from the cells for both approaches, the table below is created to compare their efficiencies.  Both approaches are within the 10 request per second limit.  

The table shows that there is a **fivefold** speedup when GNU Parallel is used appropriately

| Fetch | Link Fetching Period (seconds) | Rate of Fetching Request (request/s) | Ratio of Rate 
| ---- | :----: | :----: | :---:
| In Parallel | 0.12 | 8.30 | 5.4 
| Sequentially | 0.65 | 1.532 | 1 

## Extracting and Cleaning: Scraping Information from Form 4 Files

While all files that are labeled as Form 4 are extracted from SEC EDGAR, care must be taken that not all forms are in the same format.

As will be presented below, most Form 4 files are in XML format. The same structure allows automated extraction. However, the other files require a different approach. 

The code section below shows the total number of Form 4 files and the number of files that are not in XML format. Other files could be in `htm` or `txt` format. In addition, some of the links scraped do not even have a file extension because the script executed in the previous section identifies the `<FILENAME>` tag in the `txt` file in order to locate the XML or HTM file.

Firstly, the actual scraping of the Form 4 files from SEC EDGAR and categorisation of the files according to its extensions (xml, htm or txt). Occassionally EDGAR will block curl request as it identifies the requestor as an automation bot. As such, the HTTP code is obtained from `curl`. The script will keep requesting the same index file until the HTTP code is 200 (status 'OK').

Due to the same request blocking reason, the individual Form 4 files may not be retrieved with just one request. As such, after all index files are extracted, the script will identify file links that are incomplete, and will iterate through the list of incomplete links and request again. This process only ends when an iteration does not successfully reduce the pool of incomplete links

#### Extraction Process via Bash Script
1. 'extract_file.sh` will be run to scrape files from all years
2. After Step 1, 'recover.sh' is then run to recover the files
> This file keeps running until the pool of incomplete files remain the constant for one iteration. In each iteration of 'recover.sh', 'extract_recover.sh' will be called

In [5]:
%%bash -s "$META_SP500_RAN" "$CODE_EXTRACT"
cat "$1" | bash "$2"

Reading idx file for 1993-QTR1
Reading idx file for 1993-QTR2
Reading idx file for 1993-QTR3
Reading idx file for 1993-QTR4
Reading idx file for 1994-QTR1
Reading idx file for 1994-QTR2
Reading idx file for 1994-QTR3
Reading idx file for 1994-QTR4
Reading idx file for 1995-QTR1
Reading idx file for 1995-QTR2
Reading idx file for 1995-QTR3
Reading idx file for 1995-QTR4
Reading idx file for 1996-QTR1
Reading idx file for 1996-QTR2
Reading idx file for 1996-QTR3
Reading idx file for 1996-QTR4
Reading idx file for 1997-QTR1
Reading idx file for 1997-QTR2
Reading idx file for 1997-QTR3
Reading idx file for 1997-QTR4
Reading idx file for 1998-QTR1
Reading idx file for 1998-QTR2
Reading idx file for 1998-QTR3
Reading idx file for 1998-QTR4
Reading idx file for 1999-QTR1
Reading idx file for 1999-QTR2
Reading idx file for 1999-QTR3
Reading idx file for 1999-QTR4
Reading idx file for 2000-QTR1
Reading idx file for 2000-QTR2
Reading idx file for 2000-QTR3
Reading idx file for 2000-QTR4
Reading 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
%%bash -s "$FORM4_LINK_XML"
wc -l $1    # number of Form 4 files harvested

174387 Database/form4_data_xml.csv


2% of the files are non-XML files. Since there are three file types, three approaches are needed to scrap data. However, as the number of non-XML files is insignificant (as these extensions only exist in old Form 4 files), their extraction will be ignored, and more steps are illustrated for XML files. The extraction of the other two file types involves similar steps.

### Scrap Information from Each Form 
Form 4 files are stored in three formats: HTM, XML and txt files.  

This research scrapes the data from XML files with the help of Beautiful Soup. To begin with, the important attributes to be scraped are identified. An example of the XML file is used to explore the attributes

In [54]:
data = pd.read_csv(FORM4_LINK_XML, sep='|')

In [4]:
content = requests.get("https://www.sec.gov/Archives/edgar/data/318154/000118143104004871/rrd30540.xml").content
bs_content = bs(content, 'lxml')
print(bs_content.prettify())

<?xml version="1.0"?>
<html>
 <body>
  <ownershipdocument>
   <schemaversion>
    X0201
   </schemaversion>
   <documenttype>
    4
   </documenttype>
   <periodofreport>
    2004-01-27
   </periodofreport>
   <notsubjecttosection16>
    0
   </notsubjecttosection16>
   <issuer>
    <issuercik>
     0000318154
    </issuercik>
    <issuername>
     AMGEN INC
    </issuername>
    <issuertradingsymbol>
     AMGN
    </issuertradingsymbol>
   </issuer>
   <reportingowner>
    <reportingownerid>
     <rptownercik>
      0000904017
     </rptownercik>
     <rptownername>
      JOHNSON FRANKLIN P JR
     </rptownername>
    </reportingownerid>
    <reportingowneraddress>
     <rptownerstreet1>
      ONE AMGEN CENTER DRIVE
     </rptownerstreet1>
     <rptownerstreet2>
     </rptownerstreet2>
     <rptownercity>
      THOUSAND OAKS
     </rptownercity>
     <rptownerstate>
      CA
     </rptownerstate>
     <rptownerzipcode>
      91320-1799
     </rptownerzipcode>
     <rptownerstatedescri

A few observations are made:
- Each issuer (the reporting firm or representative) can file many Form 4
- Each reporter can file many Form 4, but the number of shares owned after the transaction is related to the reporter, not the issuer
- Each Form 4 is labeled with **an** Accession Number, which is composed of [three parts](https://www.sec.gov/edgar/searchedgar/accessing-edgar-data.htm):
    - CIK of the firm 
    - last two digits of the filing year
    - sequential count of submitted filings from that firm that year
- **Each** Form 4 file can have **multiple** transactions. This research will only focus on non-derivative transactions.
- This implies that each file has a unique Accession Number, which can assist in merging tables for exploratory analysis.

Firstly, the main table that details a Form 4 file for each entry is built. This file is stored in *form4_detail.csv*. Useful information includes

| Detail | Variable in DataFrame |
| :--- | :--- |
Trading Symbol | `trad_symbol`
Accession Number | `accession_no`
Form Number (to double-check if it is Form 4) | `form`
Reporter's name | `reporter_name`
Reporter's CIK | `reporter_cik`
Reporter's Title in the Company | `reporter_title`
Whether Trader is a Director | `is_director`
Whether Trader is an Officer | `is_officer`
Whether Trader is a Beneficial Owner (i.e. owns at least 10% of the company shares) | `is_beneficial_owner`
Whether Trader is none of the above three | `is_other`

In [2]:
def extract_xml(record):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"}
    content = requests.get(record['Link'], headers = headers).content
    bs_content = bs(content, 'lxml')
    time.sleep(0.04)
    accession_no = str(re.split('/', record['Link'])[-2])
    try:
        trad_symbol = bs_content.find("issuertradingsymbol").text
    except:
        print(record['Link'], '<issuertradingsymbol> not found. Link fails to be fetched')
        trad_symbol = ""  
    try:
        form = bs_content.find("documenttype").text
    except:
        print(record['Link'], '<documenttype> not found')
        form = np.NaN
    try:
        reporter_name = bs_content.find("rptownername").text.title()
    except:
        reporter_name = np.NaN
    try:
        reporter_cik = bs_content.find("rptownercik").text
    except:
        reporter_cik = np.NaN

    try:
        is_director = int(bs_content.find("isdirector").text)
    except:
        is_director = np.NaN
    
    try:
        is_officer = int(bs_content.find("isofficer").text)
    except:
        is_officer = np.NaN
    
    try:
        is_beneficial_owner = int(bs_content.find("istenpercentowner").text)
    except:
        is_beneficial_owner = np.NaN
    
    try:
        is_other = int(bs_content.find("isother").text)
    except:
        is_other = np.NaN
        
    try:
        reporter_title = bs_content.find("officertitle").text
    except:
        reporter_title = np.NaN
    return [trad_symbol, accession_no, form, reporter_name, reporter_cik, reporter_title, is_director, is_officer, is_beneficial_owner, is_other]

In [10]:
start = datetime.datetime.now()
batch_number = 1
for chunk in pd.read_csv(FORM4_LINK_XML, sep='|', chunksize=2000):
    chunk[['trad_symbol', 'accession_no', 'form', 'reporter_name', 'reporter_cik', 'reporter_title', 'is_director', 'is_officer', 'is_beneficial_owner', 'is_other']] = chunk.apply(extract_xml, axis = 1, result_type = 'expand')
    if batch_number == 1:
        chunk.to_csv(FORM4_DETAIL_XML, index=False, header=True, mode='w')
    else:
        chunk.to_csv(FORM4_DETAIL_XML, index=False, header=False, mode='a')
    print(f"batch {batch_number} completed")
    batch_number+=1
print("Completed execution")
end = datetime.datetime.now()
print(f'Duration {(end-start).total_seconds()}')

batch 1 completed
batch 2 completed
batch 3 completed
batch 4 completed
batch 5 completed
https://www.sec.gov/Archives/edgar/data/313616/000031361604000054/edgar.xml <issuertradingsymbol> not found. Link fails to be fetched
https://www.sec.gov/Archives/edgar/data/313616/000031361604000054/edgar.xml <documenttype> not found
batch 6 completed
batch 7 completed
batch 8 completed
batch 9 completed
https://www.sec.gov/Archives/edgar/data/49826/000004982617000048/wf-form4_148711334641755.xml <issuertradingsymbol> not found. Link fails to be fetched
https://www.sec.gov/Archives/edgar/data/49826/000004982617000048/wf-form4_148711334641755.xml <documenttype> not found
batch 10 completed
batch 11 completed
batch 12 completed
batch 13 completed
batch 14 completed
batch 15 completed
batch 16 completed
batch 17 completed
batch 18 completed
batch 19 completed
batch 20 completed
batch 21 completed
batch 22 completed
batch 23 completed
batch 24 completed
batch 25 completed
batch 26 completed
batch 27 

In [32]:
# Recover the three links that fail to be fetched
cols = ['trad_symbol', 'accession_no', 'form', 'reporter_name', 'reporter_cik', 'reporter_title', 'is_director'
        , 'is_officer', 'is_beneficial_owner', 'is_other']

links_to_retry = ['https://www.sec.gov/Archives/edgar/data/313616/000031361604000054/edgar.xml',
    'https://www.sec.gov/Archives/edgar/data/49826/000004982617000048/wf-form4_148711334641755.xml',
    'https://www.sec.gov/Archives/edgar/data/63754/000112760216057997/form4.xml']

for link in links_to_retry:
    data.loc[data['Link'] == link,cols] = extract_xml(record.squeeze())
    
# Saving these recovered file contents
data.to_csv(FORM4_DETAIL_XML, sep='|', index=False)

The extraction duration is 51880.002433 seconds, which is about 14 hours 24 minutes 40 seconds

In [36]:
data.head()

,Company,CIK,Date,Link,trad_symbol,accession_no,form,reporter_name,reporter_cik,reporter_title,is_director,is_officer,is_beneficial_owner,is_other
0,3M CO,66740,2003-07-25,https://www.sec.gov/Archives/edgar/data/66740/...,MMM,6674003000036,4.0,Grenz M Kay,1196474.0,SENIOR VICE PRESIDENT HR,0.0,1.0,0.0,0.0
1,3M CO,66740,2003-07-11,https://www.sec.gov/Archives/edgar/data/66740/...,MMM,6674003000031,4.0,Palensky Fred J,1197743.0,EXEC VP SFTY SEC & PROT SVCS,0.0,1.0,0.0,0.0
2,3M CO,66740,2003-07-25,https://www.sec.gov/Archives/edgar/data/66740/...,MMM,6674003000037,4.0,Mcnerney W James Jr,1182055.0,CHAIRMAN OF THE BOARD & CEO,1.0,1.0,0.0,0.0
3,3M CO,66740,2003-07-25,https://www.sec.gov/Archives/edgar/data/66740/...,MMM,6674003000038,4.0,Nozari M S,1197740.0,EXEC VP CONSUMER &,0.0,1.0,0.0,0.0
4,3M CO,66740,2003-07-11,https://www.sec.gov/Archives/edgar/data/66740/...,MMM,6674003000030,4.0,Belschner Ronald R,1196469.0,SENIOR VICE PRESIDENT ENGRG,0.0,1.0,0.0,0.0


### Scrap Transactions from Each Form 4
The above table only keeps information about the reporter, date and company. However, the gist -- transaction details, are not yet scraped.  

As each file can have multiple non-derivative transactions, a new table is created. To join both tables for exploratory analysis, the accession number is also kept in this table as the primary key.  

This transaction table `tx_data` stores the following details:  

| Transaction Detail | Variable in DataFrame |
:--- | :--- 
Accession Number | `accession_no`
CIK | `CIK`
Stock Type (e.g. common stock) | `stock_type`
Transaction Date | `date`
Transaction Code | `code`
Acquired (A) or Disposed (D) | `acquired`
Number of Shares Transacted | `shares_exchanged`
Number of Shares Owned After Transaction | `net_shares_owned`
Stock Price During Transaction | `stock_price`
Whether Reporter has Direct Ownership | `direct_ownership`

In [54]:
data = pd.read_csv(FORM4_DETAIL_XML, sep='|')

In [56]:
start = datetime.datetime.now()
not_found_data = pd.DataFrame(columns=['Link', 'Tag'])
not_found_tag = []
retry_links = []

batch_number = 1
for chunk in pd.read_csv(FORM4_DETAIL_XML, sep='|', chunksize = 2000):
    tx_data = pd.DataFrame(columns=['accession_no', 'CIK', 'stock_type', 'date', 'code', 'acquired', 'shares_exchanged', 'net_shares_owned', 'stock_price', 'direct_ownership'])
    for index, row in chunk.loc[:, ['accession_no', 'CIK', 'Link']].iterrows():
        
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"}
        content = requests.get(row['Link'], headers = headers).content
        bs_content = bs(content, 'lxml')

        try:
            trad_symbol = bs_content.find("issuertradingsymbol").text
        except:
            print(row['Link'], 'cannot be fetched. Retry after execution')
            retry_links.append(row['Link'])
            continue
            
        transactions = bs_content.find_all('nonderivativetransaction')
        collected_data = []
        for transaction in transactions:
            CIK = row.CIK
            accession_no = row.accession_no
            try:
                stock_type = transaction.find('securitytitle').find('value').text 
            except:
                stock_type = np.nan
                not_found_tag.append([row['Link'], "securitytitle"])

            try:
                date = datetime.datetime.strptime(transaction.find('transactiondate').find('value').text, "%Y-%m-%d").date()    # transaction date
            except:
                date = np.nan
                not_found_tag.append([row['Link'], "transactiondate"])

            try:
                code = transaction.find('transactioncode').text       # # transaction code could be M, F
            except:
                code = np.nan
                not_found_tag.append([row['Link'], "transactioncode"])

            try:
                acquired = transaction.find('transactionacquireddisposedcode').find('value').text
            except:
                acquired = np.nan
                not_found_tag.append([row['Link'], "transactionacquireddisposedcode"])

            try:
                shares_exchanged = float(transaction.find('transactionshares').find('value').text)
            except:
                shares_exchanged = np.nan
                not_found_tag.append([row['Link'], "transactionshares"])

            try:
                net_shares_owned = float(transaction.find('sharesownedfollowingtransaction').find('value').text)
            except:
                net_shares_owned = np.nan
                not_found_tag.append([row['Link'], "sharesownedfollowingtransaction"])

            try:
                stock_price = float(transaction.find('transactionpricepershare').find('value').text)    # no stock price info if shares are 
            except:
                stock_price = np.nan
                not_found_tag.append([row['Link'], "transactionpricepershare"])

            try:     
                direct_ownership = transaction.find('directorindirectownership').find('value').text     # ownership: Direct (D) or Indirect (I)
            except:
                direct_ownership = np.nan
                not_found_tag.append([row['Link'], "directorindirectownership"])

            collected_data.append([accession_no, CIK, stock_type, date, code, acquired, shares_exchanged, net_shares_owned, stock_price, direct_ownership])

        collected_data = pd.DataFrame(collected_data, columns = tx_data.columns)
        tx_data = pd.concat([tx_data, collected_data], ignore_index = True)
        
    if batch_number == 1:
        tx_data.to_csv(FORM4_TX_XML, sep='|', index=False, header=True, mode='w')
    else:
        tx_data.to_csv(FORM4_TX_XML, sep='|', index=False, header=False, mode='a')
    print(f"batch {batch_number} completed")
    batch_number+=1

not_found_data = pd.DataFrame(not_found_tag, columns = not_found_data.columns)
print("Links to retry:")
print(retry_links, "\n\n----------------------------------------------------------")
display(not_found_data)
end = datetime.datetime.now()
print("Total Execution Time (s)", (end-start).total_seconds())

batch 1 completed
batch 2 completed
batch 3 completed
batch 4 completed
batch 5 completed
batch 6 completed
batch 7 completed
batch 8 completed
batch 9 completed
batch 10 completed
batch 11 completed
batch 12 completed
batch 13 completed
batch 14 completed
batch 15 completed
batch 16 completed
batch 17 completed
batch 18 completed
batch 19 completed
batch 20 completed
batch 21 completed
batch 22 completed
batch 23 completed
batch 24 completed
batch 25 completed
batch 26 completed
batch 27 completed
batch 28 completed
batch 29 completed
batch 30 completed
batch 31 completed
batch 32 completed
batch 33 completed
batch 34 completed
batch 35 completed
batch 36 completed
batch 37 completed
batch 38 completed
batch 39 completed
batch 40 completed
batch 41 completed
batch 42 completed
batch 43 completed
batch 44 completed
batch 45 completed
batch 46 completed
batch 47 completed
batch 48 completed
batch 49 completed
batch 50 completed
batch 51 completed
batch 52 completed
batch 53 completed
ba

,Link,Tag
0,https://www.sec.gov/Archives/edgar/data/112178...,transactionpricepershare
1,https://www.sec.gov/Archives/edgar/data/875045...,transactionpricepershare
2,https://www.sec.gov/Archives/edgar/data/875045...,transactionpricepershare
3,https://www.sec.gov/Archives/edgar/data/875045...,transactionpricepershare
4,https://www.sec.gov/Archives/edgar/data/875045...,transactionpricepershare
...,...,...
8595,https://www.sec.gov/Archives/edgar/data/712515...,transactionpricepershare
8596,https://www.sec.gov/Archives/edgar/data/909832...,transactionpricepershare
8597,https://www.sec.gov/Archives/edgar/data/909832...,transactionpricepershare
8598,https://www.sec.gov/Archives/edgar/data/354190...,transactionpricepershare


Total Execution Time (s) 31265.704441


In [6]:
# Recover the links that fail to be fetched
cols = ['accession_no', 'CIK', 'stock_type', 'date', 'code', 'acquired', 'shares_exchanged', 'net_shares_owned', 
        'stock_price', 'direct_ownership']
links_to_retry = ['https://www.sec.gov/Archives/edgar/data/788784/000122320305000313/fer498.xml']
tx_data = pd.DataFrame(columns=['accession_no', 'CIK', 'stock_type', 'date', 'code', 'acquired', 'shares_exchanged', 'net_shares_owned', 'stock_price', 'direct_ownership'])
data_detail = pd.read_csv(FORM4_DETAIL_XML, sep='|')
not_found_data = pd.DataFrame(columns=['Link', 'Tag'])
not_found_tag = []
retry_links = []

for link in links_to_retry:
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"}
    content = requests.get(link, headers = headers).content
    bs_content = bs(content, 'lxml')
    row = data_detail.loc[data_detail['Link'] == link, :].squeeze()

    transactions = bs_content.find_all('nonderivativetransaction')
    collected_data = []
    for transaction in transactions:
        CIK = row.CIK
        accession_no = row.accession_no
        try:
            stock_type = transaction.find('securitytitle').find('value').text 
        except:
            stock_type = np.nan
            not_found_tag.append([link, "securitytitle"])

        try:
            date = datetime.datetime.strptime(transaction.find('transactiondate').find('value').text, "%Y-%m-%d").date()    # transaction date
        except:
            date = np.nan
            not_found_tag.append([link, "transactiondate"])

        try:
            code = transaction.find('transactioncode').text       # # transaction code could be M, F
        except:
            code = np.nan
            not_found_tag.append([link, "transactioncode"])

        try:
            acquired = transaction.find('transactionacquireddisposedcode').find('value').text
        except:
            acquired = np.nan
            not_found_tag.append([link, "transactionacquireddisposedcode"])

        try:
            shares_exchanged = float(transaction.find('transactionshares').find('value').text)
        except:
            shares_exchanged = np.nan
            not_found_tag.append([link, "transactionshares"])

        try:
            net_shares_owned = float(transaction.find('sharesownedfollowingtransaction').find('value').text)
        except:
            net_shares_owned = np.nan
            not_found_tag.append([link, "sharesownedfollowingtransaction"])

        try:
            stock_price = float(transaction.find('transactionpricepershare').find('value').text)    # no stock price info if shares are 
        except:
            stock_price = np.nan
            not_found_tag.append([link, "transactionpricepershare"])

        try:     
            direct_ownership = transaction.find('directorindirectownership').find('value').text     # ownership: Direct (D) or Indirect (I)
        except:
            direct_ownership = np.nan
            not_found_tag.append([link, "directorindirectownership"])

        collected_data.append([accession_no, CIK, stock_type, date, code, acquired, shares_exchanged, net_shares_owned, stock_price, direct_ownership])

    collected_data = pd.DataFrame(collected_data, columns = tx_data.columns)
    tx_data = pd.concat([tx_data, collected_data], ignore_index = True)

# Update the database
tx_data.to_csv(FORM4_TX_XML, sep='|', mode='a', index=False, header = False)

The extraction process takes 31265.704441seconds, which is 8 hours 41 minutes 6 seconds

In [12]:
tx_data = pd.read_csv(FORM4_TX_XML, sep='|')
tx_data.head()

,accession_no,CIK,stock_type,date,code,acquired,shares_exchanged,net_shares_owned,stock_price,direct_ownership
0,6674003000048,66740,Common Stock,2003-08-06,M,A,1052.0,1052.0,95.000,D
1,6674003000049,66740,Common Stock,2003-08-06,M,A,2560.0,15175.0,63.035,D
2,6674003000049,66740,Common Stock,2003-08-06,F,D,1163.0,14012.0,138.660,D
3,6674003000049,66740,Common Stock,2003-08-06,F,D,456.0,13556.0,138.660,D
4,6674003000049,66740,Common Stock,2003-08-06,M,A,1118.0,14674.0,90.150,D


# Exploratory Analysis

# Appendix
This section discusses about two things:
1. More complicated commands used with examples: standalone code will be run to illustrate its use
2. Specific code run to ensure quicker research process. This code is not directly related to the research focus. Rather, it serves to improve the efficiency of the whole process

## Quickening Research Process

Even though GNU Parallel has existed for more than a decade, some machines do not have it installed. Thus, the command below is run to install it. Note that GNU Parallel utility should be referenced as `/usr/local/bin/parallel` instead of parallel (which is a `gnu` version) unless an alias is set in `~/.bashrc`

In [4]:
%%bash
whereis parallel

parallel: /usr/bin/parallel /usr/local/bin/parallel /usr/share/man/man1/parallel.1.gz


In [3]:
%%bash
wget http://ftp.gnu.org/gnu/parallel/parallel-20210522.tar.bz2
sudo tar xjf parallel-20210522.tar.bz2
cd parallel-20210522
sudo ./configure && make
sudo make install
cd

checking for a BSD-compatible install... /usr/bin/install -c
checking whether build environment is sane... yes
checking for a thread-safe mkdir -p... /bin/mkdir -p
checking for gawk... no
checking for mawk... mawk
checking whether make sets $(MAKE)... yes
checking whether make supports nested variables... yes
checking whether ln -s works... yes
checking that generated files are newer than configure... done
configure: creating ./config.status
config.status: creating Makefile
config.status: creating src/Makefile
config.status: creating config.h
make  all-recursive
make[1]: Entering directory '/content/parallel-20210522'
Making all in src
make[2]: Entering directory '/content/parallel-20210522/src'
make[2]: Nothing to be done for 'all'.
make[2]: Leaving directory '/content/parallel-20210522/src'
make[2]: Entering directory '/content/parallel-20210522'
make[2]: Leaving directory '/content/parallel-20210522'
make[1]: Leaving directory '/content/parallel-20210522'
Making install in src
make[

--2021-05-25 00:36:56--  http://ftp.gnu.org/gnu/parallel/parallel-20210522.tar.bz2
Resolving ftp.gnu.org (ftp.gnu.org)... 209.51.188.20, 2001:470:142:3::b
Connecting to ftp.gnu.org (ftp.gnu.org)|209.51.188.20|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2246330 (2.1M) [application/x-bzip2]
Saving to: ‘parallel-20210522.tar.bz2’

     0K .......... .......... .......... .......... ..........  2% 1.86M 1s
    50K .......... .......... .......... .......... ..........  4% 3.74M 1s
   100K .......... .......... .......... .......... ..........  6% 35.8M 1s
   150K .......... .......... .......... .......... ..........  9%  128M 0s
   200K .......... .......... .......... .......... .......... 11% 3.76M 0s
   250K .......... .......... .......... .......... .......... 13% 60.1M 0s
   300K .......... .......... .......... .......... .......... 15%  204M 0s
   350K .......... .......... .......... .......... .......... 18%  227M 0s
   400K .......... .......... ...

Google Cloud Console is used to run `extract.sh`. However, the console quickly loses connection. To resolve this, click events are simulated by inserting the [JavaScript](https://stackoverflow.com/questions/49976573/why-google-cloud-shell-auto-disconnect-after-1-hours) code below to the browser's console

In [ ]:
%%javascript
setInterval(function() {document.elementFromPoint(500, 500).click();}, 30000);

As running the script is power-intensive, this notebook is run on Google Colab. Colab may have session timeout when it is idle even for 20 minutes. However, the whole script runs more than 2 hours. To resolve this, an iterative click simulation is performed in [JavaScript](https://stackoverflow.com/questions/54057011/google-colab-session-timeout)

In [ ]:
%%javascript
function ClickToConnect(){             // only run in Colab
    document.querySelector('#top-toolbar > colab-connect-button').shadowRoot.querySelector('#connect').click();
    console.log("Connecting")
}
setInterval(ClickToConnect, 60*1000)
// clearInterval(ClickToConnect)  // to stop the click event

<IPython.core.display.Javascript object>

`tee` can print the output to files and stdin simulatneously. This is useful for executing the actual code while showing some content in the file for easier understanding. However, one can argue that printing to a file and then using `head` could achieve the same results.  

Is there any difference in terms of time? Will using `tee` slow the output process? The code below outputs the statistics into a file, with the size of the file content as the variable

In [ ]:
%%bash
echo "Approach 1: without saving to a file, use tee to send output to head" >tee_head_time.csv
echo "Number of Rows: Duration (milliseconds)" >>tee_head_time.csv
for ((row_size = 10; row_size < 1000000000; row_size*=10)) do
    start=$(($(date +%s%N)/1000000));
    echo "digit_1,mul2,digit_2,digit_3,digit_4,digit_5,mod3,mod5,doubledigit_1,digit_6,digit_7,digit_8,mod4,doubledigit_2,digit_9,digit_10,digit_11,mod7,triple,doubledigit_3,digit12,digit13,digit14"
    for i in $(seq 1 1 $row_size); do 
        tmp=$(($i%10));
        echo $tmp,$(($(($i*2))%10)),$tmp,$tmp,$tmp,$tmp,$(($tmp % 3)),$(($tmp % 5)),$tmp$tmp,$tmp,$tmp,$tmp,$(($tmp % 4)),$tmp$tmp,$tmp,$tmp,$tmp,$(($tmp % 7)),$tmp$tmp$tmp,$tmp$tmp,$tmp,$tmp,$(($tmp % 4)); 
    done | tee | head 
    end=$(($(date +%s%N)/1000000))
    echo $row_size: $(($end-$start)) >>tee_head_time.csv
done;

echo "Approach 2: save output in a file, then use head utility" >>tee_head_time.csv
echo "Number of Rows: Duration (milliseconds)" >>tee_head_time.csv
for ((row_size = 10; row_size < 1000000000; row_size*=10)) do
    start=$(($(date +%s%N)/1000000));
    echo "digit_1,mul2,digit_2,digit_3,digit_4,digit_5,mod3,mod5,doubledigit_1,digit_6,digit_7,digit_8,mod4,doubledigit_2,digit_9,digit_10,digit_11,mod7,triple,doubledigit_3,digit12,digit13,digit14" >test.csv
    for i in $(seq 1 1 $row_size); do 
        tmp=$(($i%10));
        echo $tmp,$(($(($i*2))%10)),$tmp,$tmp,$tmp,$tmp,$(($tmp % 3)),$(($tmp % 5)),$tmp$tmp,$tmp,$tmp,$tmp,$(($tmp % 4)),$tmp$tmp,$tmp,$tmp,$tmp,$(($tmp % 7)),$tmp$tmp$tmp,$tmp$tmp,$tmp,$tmp,$(($tmp % 4)); 
    done >>test.csv
    head test.csv
    end=$(($(date +%s%N)/1000000))
    echo $row_size: $(($end-$start)) >>tee_head_time.csv
    rm test.csv;
done;

# getting milliseconds [https://stackoverflow.com/questions/16548528/command-to-get-time-in-milliseconds]

digit_1,mul2,digit_2,digit_3,digit_4,digit_5,mod3,mod5,doubledigit_1,digit_6,digit_7,digit_8,mod4,doubledigit_2,digit_9,digit_10,digit_11,mod7,triple,doubledigit_3,digit12,digit13,digit14
1,2,1,1,1,1,1,1,11,1,1,1,1,11,1,1,1,1,111,11,1,1,1
2,4,2,2,2,2,2,2,22,2,2,2,2,22,2,2,2,2,222,22,2,2,2
3,6,3,3,3,3,0,3,33,3,3,3,3,33,3,3,3,3,333,33,3,3,3
4,8,4,4,4,4,1,4,44,4,4,4,0,44,4,4,4,4,444,44,4,4,0
5,0,5,5,5,5,2,0,55,5,5,5,1,55,5,5,5,5,555,55,5,5,1
6,2,6,6,6,6,0,1,66,6,6,6,2,66,6,6,6,6,666,66,6,6,2
7,4,7,7,7,7,1,2,77,7,7,7,3,77,7,7,7,0,777,77,7,7,3
8,6,8,8,8,8,2,3,88,8,8,8,0,88,8,8,8,1,888,88,8,8,0
9,8,9,9,9,9,0,4,99,9,9,9,1,99,9,9,9,2,999,99,9,9,1
0,0,0,0,0,0,0,0,00,0,0,0,0,00,0,0,0,0,000,00,0,0,0
digit_1,mul2,digit_2,digit_3,digit_4,digit_5,mod3,mod5,doubledigit_1,digit_6,digit_7,digit_8,mod4,doubledigit_2,digit_9,digit_10,digit_11,mod7,triple,doubledigit_3,digit12,digit13,digit14
1,2,1,1,1,1,1,1,11,1,1,1,1,11,1,1,1,1,111,11,1,1,1
2,4,2,2,2,2,2,2,22,2,2,2,2,22,2,2,2,2,222,22,2,2,2
3,6,3,3,3,3,